In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

In [2]:
# Check the Python version
import sys
if sys.version.startswith("3."):
  print("You are running Python 3. Good job :)")
else:
  print("This notebook requires Python 3.\nIf you are using Google Colab, go to Runtime > Change runtime type and choose Python 3.")

You are running Python 3. Good job :)


In [ ]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False
if IN_COLAB:
  # Clone the entire repo to access the files.
  !git clone -l -s https://github.com/epfml/OptML_course.git cloned-repo
  %cd cloned-repo/labs/ex02/template/

# Load the data

In [3]:
import datetime
from helpers import *

height, weight, gender = load_data(sub_sample=False, add_outlier=False)
x, mean_x, std_x = standardize(height)
b, A = build_model_data(x, weight)

In [4]:
print('Number of samples n = ', b.shape[0])
print('Dimension of each sample d = ', A.shape[1])

Number of samples n =  10000
Dimension of each sample d =  2


# Least Squares Estimation
Least squares estimation is one of the fundamental machine learning algorithms. Given an $ n \times d $ matrix $A$ and a $ n \times 1$ vector $b$, the goal is to find a vector $x \in \mathbb{R}^d$ which minimizes the objective function $$f(x) = \frac{1}{2n} \sum_{i=1}^{n} (a_i^\top x - b_i)^2 = \frac{1}{2n} \|Ax - b\|^2 $$

In this exercise, we will try to fit $x$ using Least Squares Estimation. 

One can see the function is $L$ smooth with $L =\frac1n\|A^T A\|  = \frac1n\|A\|^2$ (Lemma 2.3 for the first equality, and a few manipulations for the second).

# Computing the Objective Function
Fill in the `calculate_objective` function below:

In [13]:
def calculate_objective(Axmb):
    """Calculate the mean squared error for vector Axmb = Ax - b."""
    # ***************************************************
    # INSERT YOUR CODE HERE
    # TODO: compute mean squared error
    return 1/2*np.mean(Axmb**2)
    # ***************************************************
    raise NotImplementedError

# Compute smoothness constant $L$

To compute the spectral norm of A you can use np.linalg.norm(A, 2)

In [7]:
def calculate_L(b, A):
    """Calculate the smoothness constant for f"""
    # ***************************************************
    # INSERT YOUR CODE HERE
    # TODO: compute ||A.T*A||
    temp=np.linalg.norm(A.T@A,2)
    # ***************************************************
    #raise NotImplementedError
    # ***************************************************
    # INSERT YOUR CODE HERE
    # TODO: compute L = smoothness constant of f
    L=1/A.shape[0]*temp
    # ***************************************************
    #raise NotImplementedError
    return L

# Gradient Descent

Please fill in the functions `compute_gradient` below:

In [11]:
def compute_gradient(b, A, x):
    """Compute the gradient."""
    # ***************************************************
    # INSERT YOUR CODE HERE
    # TODO: compute gradient and objective
    Axmb=A@x-b
    grad=A.T@Axmb/A.shape[0]
    # ***************************************************
    #raise NotImplementedError
    return grad, Axmb

Please fill in the functions `gradient_descent` below:

In [9]:
def gradient_descent(b, A, initial_x, max_iters, gamma):
    """Gradient descent algorithm."""
    # Define parameters to store x and objective func. values
    xs = [initial_x]
    objectives = []
    x = initial_x
    for n_iter in range(max_iters):
        # ***************************************************
        # INSERT YOUR CODE HERE
        # TODO: compute gradient and objective function
        grad,Axmb=compute_gradient(b,A,x)
        obj=calculate_objective(Axmb)
        # ***************************************************
        #raise NotImplementedError
        # ***************************************************
        # INSERT YOUR CODE HERE
        # TODO: update x by a gradient descent step
        # ***************************************************
        x=x-gamma*grad
        #raise NotImplementedError
        # store x and objective function value
        xs.append(x)
        objectives.append(obj)
        print("Gradient Descent({bi}/{ti}): objective={l}".format(
              bi=n_iter, ti=max_iters - 1, l=obj))

    return objectives, xs

Test your gradient descent function with a naive step size through gradient descent demo shown below:

In [14]:
# from gradient_descent import *
from plots import gradient_descent_visualization

# Define the parameters of the algorithm.
max_iters = 50

gamma = 0.1

# Initialization
x_initial = np.zeros(A.shape[1])

# Start gradient descent.
start_time = datetime.datetime.now()
gradient_objectives_naive, gradient_xs_naive = gradient_descent(b, A, x_initial, max_iters, gamma)
end_time = datetime.datetime.now()

# Print result
exection_time = (end_time - start_time).total_seconds()
print("Gradient Descent: execution time={t:.3f} seconds".format(t=exection_time))

Gradient Descent(0/49): objective=2792.2367127591674
Gradient Descent(1/49): objective=2264.6350560300057
Gradient Descent(2/49): objective=1837.2777140793814
Gradient Descent(3/49): objective=1491.1182670993776
Gradient Descent(4/49): objective=1210.729115045574
Gradient Descent(5/49): objective=983.6139018819921
Gradient Descent(6/49): objective=799.6505792194914
Gradient Descent(7/49): objective=650.6402878628656
Gradient Descent(8/49): objective=529.9419518639996
Gradient Descent(9/49): objective=432.1762997049176
Gradient Descent(10/49): objective=352.98612145606046
Gradient Descent(11/49): objective=288.8420770744868
Gradient Descent(12/49): objective=236.8854011254118
Gradient Descent(13/49): objective=194.80049360666106
Gradient Descent(14/49): objective=160.711718516473
Gradient Descent(15/49): objective=133.0998106934208
Gradient Descent(16/49): objective=110.73416535674836
Gradient Descent(17/49): objective=92.6179926340438
Gradient Descent(18/49): objective=77.9438927286531

Time Visualization

In [15]:
from ipywidgets import IntSlider, interact
from grid_search import *

def plot_figure(n_iter):
    # Generate grid data for visualization (parameters to be swept and best combination)
    grid_x0, grid_x1 = generate_w(num_intervals=10)
    grid_objectives = grid_search(b, A, grid_x0, grid_x1)
    obj_star, x0_star, x1_star = get_best_parameters(grid_x0, grid_x1, grid_objectives)
    
    fig = gradient_descent_visualization(
        gradient_objectives_naive, gradient_xs_naive, grid_objectives, grid_x0, grid_x1, mean_x, std_x, height, weight, n_iter)
    fig.set_size_inches(10.0, 6.0)

interact(plot_figure, n_iter=IntSlider(min=1, max=len(gradient_xs_naive)))

interactive(children=(IntSlider(value=1, description='n_iter', max=51, min=1), Output()), _dom_classes=('widge…

<function __main__.plot_figure(n_iter)>

Try doing gradient descent with a better learning rate

In [16]:
# Define the parameters of the algorithm.
max_iters = 50

# ***************************************************
# INSERT YOUR CODE HERE
# TODO: a better learning rate using the smoothness of f
# ***************************************************
gamma = 1/calculate_L(b,A)
#raise NotImplementedError

# Initialization
x_initial = np.zeros(A.shape[1])

# Start gradient descent.
start_time = datetime.datetime.now()
gradient_objectives, gradient_xs = gradient_descent(b, A, x_initial, max_iters, gamma)
end_time = datetime.datetime.now()

# Print result
exection_time = (end_time - start_time).total_seconds()
print("Gradient Descent: execution time={t:.3f} seconds".format(t=exection_time))

Gradient Descent(0/49): objective=2792.2367127591674
Gradient Descent(1/49): objective=15.3858878688294
Gradient Descent(2/49): objective=15.3858878688294
Gradient Descent(3/49): objective=15.3858878688294
Gradient Descent(4/49): objective=15.3858878688294
Gradient Descent(5/49): objective=15.3858878688294
Gradient Descent(6/49): objective=15.3858878688294
Gradient Descent(7/49): objective=15.3858878688294
Gradient Descent(8/49): objective=15.3858878688294
Gradient Descent(9/49): objective=15.3858878688294
Gradient Descent(10/49): objective=15.3858878688294
Gradient Descent(11/49): objective=15.3858878688294
Gradient Descent(12/49): objective=15.3858878688294
Gradient Descent(13/49): objective=15.3858878688294
Gradient Descent(14/49): objective=15.3858878688294
Gradient Descent(15/49): objective=15.3858878688294
Gradient Descent(16/49): objective=15.3858878688294
Gradient Descent(17/49): objective=15.3858878688294
Gradient Descent(18/49): objective=15.3858878688294
Gradient Descent(19/

Time visualization with a better learning rate

In [17]:
def plot_figure(n_iter):
    # Generate grid data for visualization (parameters to be swept and best combination)
    grid_x0, grid_x1 = generate_w(num_intervals=10)
    grid_objectives = grid_search(b, A, grid_x0, grid_x1)
    obj_star, x0_star, x1_star = get_best_parameters(grid_x0, grid_x1, grid_objectives)
    
    fig = gradient_descent_visualization(
        gradient_objectives, gradient_xs, grid_objectives, grid_x0, grid_x1, mean_x, std_x, height, weight, n_iter)
    fig.set_size_inches(10.0, 6.0)

interact(plot_figure, n_iter=IntSlider(min=1, max=len(gradient_xs)))

interactive(children=(IntSlider(value=1, description='n_iter', max=51, min=1), Output()), _dom_classes=('widge…

<function __main__.plot_figure(n_iter)>

# Loading more complex data
The data is taken from https://archive.ics.uci.edu/ml/datasets/Concrete+Compressive+Strength 

In [18]:
data = np.loadtxt("Concrete_Data.csv",delimiter=",")

A = data[:,:-1]
b = data[:,-1]
A, mean_A, std_A = standardize(A)

In [19]:
print('Number of samples n = ', b.shape[0])
print('Dimension of each sample d = ', A.shape[1])

Number of samples n =  1030
Dimension of each sample d =  8


# Running gradient descent

## Assuming bounded gradients
Assume we are moving in a bounded region $\|x\| \leq 25$ containing all iterates (and we assume $\|x-x^\star\| \leq 25$ as well, for simplicity). Then by $\nabla f(x) = \frac{1}{n}A^\top (Ax - b)$, one can see that $f$ is Lipschitz over that bounded region, with Lipschitz constant $\|\nabla f(x)\| \leq \frac{1}{n} (\|A^\top A\|\|x\| + \|A^\top b\|)$

In [22]:
# ***************************************************
# INSERT YOUR CODE HERE
# TODO: Compute the bound on the gradient norm
# ***************************************************
grad_norm_bound = 1/A.shape[0]*(np.linalg.norm(A.T@A,2)*25+np.linalg.norm(A.T@b,2))
#raise NotImplementedError

Fill in the learning rate assuming bounded gradients

In [23]:
max_iters = 50

# ***************************************************
# INSERT YOUR CODE HERE
# TODO: Compute learning rate based on bounded gradient
# ***************************************************
gamma = 25/(grad_norm_bound*np.sqrt(max_iters))
#raise NotImplementedError

# Initialization
x_initial = np.zeros(A.shape[1])

# Start gradient descent.
start_time = datetime.datetime.now()
bd_gradient_objectives, bd_gradient_xs = gradient_descent(b, A, x_initial, max_iters, gamma)
end_time = datetime.datetime.now()


# Print result
exection_time = (end_time - start_time).total_seconds()
print("Gradient Descent: execution time={t:.3f} seconds".format(t=exection_time))

# Averaging the iterates as is the case for bounded gradients case
bd_gradient_objectives_averaged = []
for i in range(len(bd_gradient_xs)):
    if i > 0:
        bd_gradient_xs[i] = (i * bd_gradient_xs[i-1] + bd_gradient_xs[i])/(i + 1)
    grad, err = compute_gradient(b, A, bd_gradient_xs[i])
    obj = calculate_objective(err)
    bd_gradient_objectives_averaged.append(obj)

Gradient Descent(0/49): objective=780.8686016504854
Gradient Descent(1/49): objective=771.9451380356
Gradient Descent(2/49): objective=764.0577873681689
Gradient Descent(3/49): objective=757.0806435792373
Gradient Descent(4/49): objective=750.9036210736357
Gradient Descent(5/49): objective=745.4304063496157
Gradient Descent(6/49): objective=740.5766831862035
Gradient Descent(7/49): objective=736.2685935205593
Gradient Descent(8/49): objective=732.4414016205171
Gradient Descent(9/49): objective=729.0383338025285
Gradient Descent(10/49): objective=726.0095698886381
Gradient Descent(11/49): objective=723.3113659504753
Gradient Descent(12/49): objective=720.9052907468908
Gradient Descent(13/49): objective=718.7575607023603
Gradient Descent(14/49): objective=716.8384603602686
Gradient Descent(15/49): objective=715.1218370326862
Gradient Descent(16/49): objective=713.5846599014745
Gradient Descent(17/49): objective=712.206635142504
Gradient Descent(18/49): objective=710.9698697774038
Gradien

## Gradient descent using smoothness
Fill in the learning rate using smoothness of the function

In [ ]:
max_iters = 50


# ***************************************************
# INSERT YOUR CODE HERE
# TODO: a better learning rate using the smoothness of f
# ***************************************************
gamma = 
raise NotImplementedError

# Initialization
x_initial = np.zeros(A.shape[1])

# Start gradient descent.
start_time = datetime.datetime.now()
gradient_objectives, gradient_xs = gradient_descent(b, A, x_initial, max_iters, gamma)
end_time = datetime.datetime.now()

# Print result
exection_time = (end_time - start_time).total_seconds()
print("Gradient Descent: execution time={t:.3f} seconds".format(t=exection_time))

## Plotting the Evolution of the Objective Function

In [ ]:
plt.figure(figsize=(8, 8))
plt.xlabel('Number of steps')
plt.ylabel('Objective Function')
#plt.yscale("log")
plt.plot(range(len(gradient_objectives)), gradient_objectives,'r', label='gradient descent with 1/L stepsize')
plt.plot(range(len(bd_gradient_objectives)), bd_gradient_objectives,'b', label='gradient descent assuming bounded gradients')
plt.plot(range(len(bd_gradient_objectives_averaged)), bd_gradient_objectives_averaged,'g', label='gradient descent assuming bounded gradients with averaged iterates')
plt.legend(loc='upper right')
plt.show()